In [1]:
import numpy as np
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
with open('chinese_stopwords.txt','r',encoding='utf-8')as file:
    #加载停用词
    stopwords = [i[:-1] for i in file.readlines()]
    #数据加载
    news = pd.read_csv('sqlResult.csv',encoding='_gb18030')
    print(news.shape)
    print(news.head())

(89611, 7)
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...   
1  骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...   
2  此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...   
3    这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n   
4  （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...   

                                             feature  \
0  {"type":"科技","site":"cnbeta","commentNum":"37"...   
1  {"type":"科技","site":"cnbeta","commentNum":"15"...   
2  {"type":"科技","site":"cnbeta","commentNum":"18"...   
3  {"type":"国际新闻","site":"环球","commentNum":"0","j...   
4  {"type":"新闻","site":"网易热门","commentNum":"978",...   

                          

In [7]:
#处理缺失值
print(news[news.content.isna()].head())
news = news.dropna(subset=['content'])
print(news.shape)
#分词
def split_text(text):
    text = text.replace(' ','')
    text = text.replace('\n','')
    text2 = jieba.cut(text.strip())
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result
print(news.iloc[0].content)
print(split_text(news.iloc[0].content))
import pickle,os
if not os.path.exists('corpus.pkl'):

    corpus = list(map(split_text,[str(i) for i in news.content]))
    print(corpus[0])
    print(len(corpus))
    print(corpus[1])
    with open('corpus.pkl','wb')as file:
        pickle.dump(corpus,file)
else:
    #调用上次的处理结果
    with open('corpus.pkl','rb') as file:
        corpus = pickle.load(file)
    

Empty DataFrame
Columns: [id, author, source, content, feature, title, url]
Index: []
(87054, 7)
此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。
MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。
当然，关于MIUI 9的确切信息，我们还是等待官方消息。

 当然 MIUI9 确切 信息 等待 官方消息 更新换代  发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 
 当然 MIUI9 确切 信息 等待 官方消息 更新换代  发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 
87054
 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台30% PCB 空间 找到 一种 适合 桌面 平台 兼顾 性能 功耗 完美 方案 


In [10]:
#计算corpus的TF-IDF矩阵
countvectorizer = CountVectorizer(encoding='gb18030',min_df=0.015)
tfidftransformer = TfidfTransformer()
countvector = countvectorizer.fit_transform(corpus)
tfidf = tfidftransformer.fit_transform(countvector)
print(tfidf.shape)

(87054, 884)


In [12]:
# 标记是否是自己的新闻
label = list(map(lambda source: 1 if '新华' in str(source) else 0,news.source ))
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
#数据切分
X_train, X_test, y_train, y_test = train_test_split(tfidf.toarray(), label,test_size=0.3,random_state=33)
clf = MultinomialNB()
#分类器使用fit 和predict
clf.fit(X_train,y_train)
prediction = clf.predict(tfidf.toarray())
labels = np.array(label)

compare_news_index = pd.DataFrame({'prediction': prediction,'labels':labels})
#计算所有可以文章的idnex
copy_news_index = compare_news_index[(compare_news_index['prediction']== 1)&(compare_news_index['labels']== 0)].index
#计算所有新华文章的index
xinhuashe_news_index = compare_news_index[(compare_news_index['labels']==1)].index
print('可疑文章数',len(copy_news_index))



可疑文章数 2828


In [13]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfidf.toarray())
if not os.path.exists('label.pkl'):
    
    #使用Kmeans,对全部文档聚类
    kmeans = KMeans(n_clusters=25)
    k_labels = kmeans.fit_predict(scaled_array)
    with open('label.pkl','wb')as file:
        pickle.dump(k_labels,file)
    print('k_labels.shape',k_labels.shape)
else:
    with open('label.pkl','rb')as file:
        k_labels = pickle.load(file)

k_labels.shape (87054,)


In [14]:
#创建id_class
if not os.path.exists('id_class.pkl'):
    
    #创建id_class
    id_class = {index:class_ for index,class_ in enumerate(k_labels)}
    k_labels = kmeans.fit_predict(scaled_array)
    with open('id_class.pkl','wb')as file:
        pickle.dump(id_class,file)
else:
    with open('label.pkl','rb')as file:
        id_class= pickle.load(file)

In [16]:
if not os.path.exists('class_id.pkl'):
    from collections import defaultdict
    #创建class_id
    class_id = defaultdict(set)
    for index,class_ in id_class.items():
        #只统计新华社发布的class_id
        if index in xinhuashe_news_index.tolist():
            class_id[class_].add(index)
   
    with open('class_id.pkl','wb')as file:
        pickle.dump(class_id,file)
else:
    with open('class_id.pkl','rb')as file:
        class_id = pickle.load(file)

In [ ]:
#想找相似文本
def find_similar_text(cpindex,top=10):
    #只在新华社发布文章中查找
    dist_dict = {i:cosine_similarity(tfidf[cpindex], tfidf[i]) for i in class_id[id_class[cpindex]]}
    #从大到小进行排序
    return sorted(dist_dict.items(), key=lambda x:x[1][0],reverse=True)[:top]
cpindex = 3352
similar_list = find_similar_text(cpindex)
print(similar_list)
print('怀疑抄袭:\n',news.iloc[cpindex].content)
#找一片相似的原文
similar2 = similar_list[0][0]
print('相似原文\n',news.iloc[similar2].content)

import editdistance
print('编辑距离:',editdistance.eval(corpus[cpindex],corpus[similar2]))